<hr style="border-width:4px; border-color:coral"/>

# Creating fractals
<hr style="border-width:4px; border-color:coral"/>

Fractals are fascinating images that can be computed using a very simple fixed-point type iteration.  And because the iterations run independently, algorithms for fractal generation parallelize very naturally.  The code below illustrate the Julia fractal.  You can read more about Julia fractals on this <a href="https://en.wikipedia.org/wiki/Julia_set">Wikipedia page</a>.

Your assignment is to write an MPI/CUDA code to generate the fractal, and write out the solution and plot it in a notebook.  Ultimately, you can create image files for a continuum of Julia sets, and plot the animation of the sets in teh notebook as well.  

In [ ]:
%matplotlib notebook
%pylab

from matplotlib import colors
import warnings

The basic algorithm for computing Julia fractals is an iterative scheme, based on the following idea. 

Given a number $z_0 = x  + iy$ in the complex plane, generate iterates

\begin{equation*}
z_{k+1} = g(z_k)
\end{equation*}

where $g(z)$ is a prescribed function.  For example, the Julia fractal below uses $g(z) = z^2 + c$ where $c$ is a complex number.   

We generate these iterates for every point $z_0$ in a region $D$ in the complex plane 
(e.g. $z \in D = [-2,2]x[-2,2]$).  Then, we color each point in the region $D$ based on how quickly the sequence $\{z_{k}\}$ diverges.  To determine divergence, we measure $|z_k| = \sqrt{x^2 + y^2}$.  The sequence $z_k$ is said to have an "escape time" $K(z)$ if $|z_k| > \rho$, for all $k > K(z)$, for some $\rho \in R$ (fixed for all $z$ in $D$). If, after some predetermined $K_{max}$, we have $|z_k| < \rho, k \ge K_{max}$, we assign the escape time as $K(z_k) = \infty$.  For these values, the sequence $z_k$ does not diverge.  

Once every point $z \in D$ has an escape time $K(z)$, we can then color the domain by assigning colors to each integer value $K(z)$.  

In [ ]:
def julia_set(ax,bx,ay,by, Nx, Ny, kmax,c):
    
    # Generate points in complex planes D
    xe = linspace(ax,bx, Nx+1).astype(float32)
    ye = linspace(ay,by, Ny+1).astype(float32)
    dx = (bx-ax)/N
    dy = (by-ay)/N
    xc = xe[:-1] + dx/2            
    yc = ye[:-1] + dy/2            

    # Set of initial values Z0
    zk = xc + yc[:, None] * 1j
    
    # Constant needed for Julia fractal : g(z) = z^2 + c
    C = zeros_like(zk) + c
    
    rho = 2.0
    
    # Vectorize the computation of g(z); Use 
    escape_time = zeros_like(zk,dtype=int)
    for n in range(kmax):
        I = logical_and(greater(abs(zk), rho),equal(escape_time,0))
        escape_time[I] = n
        notI = not_equal(I,True)
        zk[notI] = zk[notI]**2 + C[notI]
    escape_time[escape_time == 0] = 0   # These sequences didn't diverge;  use 0 here instead of infinity. 
    return zk, escape_time

In [ ]:
%%time

N = 1024
kmax = 400

# Domain D is centered at (xc,yc) and has width 'domain_width'

# Zoomed fractal
domain_width = 0.01
xc = 1.15657568
yc = -0.1002331

# Full fractal
domain_width = 4
xc = 0
yc = 0

# dimensions of D
ax, bx = xc - domain_width/2, xc + domain_width/2
ay, by = yc - domain_width/2, yc + domain_width/2

c = -0.8+0.156*1j

warnings.simplefilter("ignore")   # Suppress overflow run time warnings
Z, M = julia_set(ax, bx, ay, by, N, N, kmax,c)

## Plot results

In [ ]:
%%time

dpi = 256         # Figure might not show up in the notebook if dpi is too high;  but PNG file will be created (below).
width = N/dpi     # Final figure size (in pixels) :  dpi*[width,height]
height = N/dpi
fig = figure(figsize=(width, height), dpi=dpi)
fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=False, aspect=1)

# Shaded rendering
light = colors.LightSource(azdeg=315, altdeg=10)
M_plot = light.shade(M, cmap=plt.cm.plasma, vert_exag=1.5,
                norm=colors.PowerNorm(0.3), blend_mode='hsv')
imshow(M_plot, extent=[ax,bx,ay,by],interpolation='bicubic',origin='lower')

savefig("fractal.png")
show()